<a href="https://colab.research.google.com/github/piusAI/DeepLearning_Basic/blob/main/Chap05_05_Learn_Implement_from_Back_Propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BackPropagation

### Prepration TwoLayerNet

#### def Function

In [43]:
import numpy as np
import keras.datasets.mnist as mnist

mnist.load_data()

def sigmoid(x):
  return 1 / 1+ np.exp(-x)

def softmax(a):
  if a.ndim==2:
    a = a - np.max(a, axis=1, keepdims=1)
    y = np.exp(a) / np.sum(np.exp(a), axis=1, keepdims=1)
  else:
    a = a - np.max(a)
    y = np.exp(a) / np.sum(np.exp(a))
  return y

def CEE(y, t):
  if y.ndim==1:
    y.reshape = (y.shape[0], -1)
    t.reshape = (t.shape[0], -1)
  batch_size = y.shape[0]
  loss = np.log(np.sum( y[np.arange(batch_size),t] )) /batch_size
  return loss

#### Def Code - Relu, Affine, SoftmaxWithLoss - 따라침

In [44]:
import numpy as np

class Relu:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x<=0)
    out = x.copy()
    out[self.mask] = 0
    return out

  def backward(self, dout):
    dout[self.mask]=0
    dx = dout

    return dx

class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b

    self.x = None
    self.original_x_shape = None

    self.dW = None
    self.db = None

  def forward(self, x):
    self.original_x_shape = x.shape
    x = x.reshape(x.shape[0], -1)

    self.x = x
    out = np.dot(self.x, self.W) + self.b

    return out

  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)

    dx = dx.reshape(*self.original_x_shape)
    return dx

class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self. loss = CEE(self.y, self.t)

    return self.loss

  def backward(self, dout = 1):
    batch_size = self.t.shape[0]
    if self.t.size == self.y.size:
      dx = (self.y - self.t) / batch_size

    else :
      dx = self.y.copy()
      dx[np.arange(batch_size), self.t] -=1
      dx = dx/ batch_size
    return dx

위 코드 직접 구현 추후 해보기

### Implement TwoLayer Net

In [45]:

from collections import OrderedDict

class TwolayerNet():
  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    self.params={}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

    self.lastlayers = SoftmaxWithLoss()

  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forwad(x)
    return x

  def loss(self, x,t):
    y = self.x.predict(x)
    return self.lastlayers.forward(y,t)


✅ 정리 요약

-

1.   역전파시, 순서가 중요하기떄문에 OrderedDiction으로!


➡ Affine1, Relu1, Affine2 이 순서가 중요!

2.   predict는 Softmax이전까지의 Forward(순전파)

3.   forward는 Affine.forward, Relu.forward의 순전파 Module

In [46]:
tl = TwolayerNet(input_size = 784, hidden_size = 50, output_size = 10)

print(tl.layers.values())
print(tl.layers.keys())

odict_values([<__main__.Affine object at 0x79a2b2f86250>, <__main__.Relu object at 0x79a2af58ee50>, <__main__.Affine object at 0x79a2af7809d0>])
odict_keys(['Affine1', 'Relu1', 'Affine2'])


=> Key, Value를 확인할 수 있다

#### OrderedDictionary?

In [47]:
from collections import OrderedDict

a = {'Pius' : "Ai", "JGAi" : "Soccer", "SaaS" : "Service"}
b = {"JGAi" : "Soccer", "SaaS" : "Service",'Pius' : "Ai" }

c=OrderedDict(a)
d=OrderedDict(b)

print(a==b)
print(c==d)

True
False


- Dictionary는 순서에 구애 받지 않는 Key-Value.
- OrderDictionary는 순서에 구애 받는다.